In [ ]:
%reset

In [ ]:
from os import listdir
from os.path import exists
from importlib import reload
import numpy as np
import pandas as pd
import pyxdf
import mne
from utils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
import time
import datetime
from datetime import datetime, timezone
import pickle
import plotly.express as px

print('Imports done...')

In [ ]:
# Load classification results:
df = pd.read_csv('classification_df.csv', index_col=0)

In [ ]:
# Calculate CI for all subjects with the same '5-point' and 'Type' column for each timepoint.
cols = ['Timepoint', 'Mean_accuracy', 'Type', '5-point', 'Upper', 'Lower']
confidence = .95
df_mean = pd.DataFrame(columns=cols)
for type in df['Type'].unique():
    for windowed in df['5-point'].unique():
        for tp in df[(df['Type'] == type) & (df['5-point'] == windowed)]['Timepoint'].unique():
            # Get average overall subjects for each timepoint:
            accs = np.array(df[(df['Type'] == type) & (df['5-point'] == windowed) & (df['Timepoint'] == tp)]['Accuracy'])
            mean = accs.mean()

            # Bootstrapping for confidence interval:
            values = [np.random.choice(accs,size=len(accs),replace=True).mean() for i in range(1000)]
            lower, upper = np.percentile(values,[100*(1-confidence)/2,100*(1-(1-confidence)/2)])

            df_mean.loc[len(df_mean)] = [tp, mean, type, windowed, upper, lower]

In [ ]:
for type in df_mean['Type'].unique():
    leg = []
    plt.figure(figsize=(10,6))
    for windowed in df_mean['5-point'].unique():
        # Extract tps, means and confidences to plot:
        timepoints = list(df_mean[(df_mean['Type'] == type) & (df_mean['5-point'] == windowed)]['Timepoint'])
        means = list(df_mean[(df_mean['Type'] == type) & (df_mean['5-point'] == windowed)]['Mean_accuracy'])
        uppers = list(df_mean[(df_mean['Type'] == type) & (df_mean['5-point'] == windowed)]['Upper'])
        lowers = list(df_mean[(df_mean['Type'] == type) & (df_mean['5-point'] == windowed)]['Lower'])

        if windowed:
            txt = '5-point windowed'
        else:
            txt = 'single timepoint'

        leg.append(txt + ' mean')
        leg.append(txt + ' 95% confidence interval')

        plt.plot(timepoints, means)
        plt.fill_between(timepoints, lowers, uppers, alpha=0.1)

    plt.title(f'Classification accuracy for {type}')
    plt.legend(leg, loc='best')
    plt.ylim([0,1])
    plt.show()